In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import retro

import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import random
import os

#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

Using TensorFlow backend.


In [3]:
#path variables
#game_url = "chrome://dino"
#chrome_driver_path = "/usr/local/bin/chromedriver"
#chrome_driver_path = "./chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

#scripts
#create id for canvas for faster selection from DOM
#init_script = "document.getElementsByClassName('runner-canvas')[0].id = 'runner-canvas'"

#get image from canvas
#getbase64Script = "canvasRunner = document.getElementById('runner-canvas'); \
#return canvasRunner.toDataURL().substring(22)"

In [4]:
class Game:
    def __init__(self,env_name):
        self.env = retro.make(game=env_name)
    def restart(self):
        obs = self.env.reset()
        return obs
    def render(self):
        self.env.render()
        
    def Run(self):
        act = [0,0,0,0,0,0,0,1,0]
        return act
    def Jump(self):
        act = [0,0,0,0,0,0,0,1,1]
        return act
    def Stop(self):
        act = [0,0,0,0,0,0,1,1,0]
        return act
    def get_score(self, act):
        obs, rew, done, info = self.env.step(act)
        
        return obs, rew, done, info
    
    def get_action_sample(self):
        act = self.env.action_space.sample()
        return act
    
    def end(self):
        self.env.close()

In [5]:
class clown_agent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game
        
    def Run(self):
        self._game.Run()
    def Jump(self):
        self._game.Jump()
    def Stop(self):
        self._game.Stop()

In [6]:
class Game_state_:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        
        image, reward, is_over, info = self._game.get_score(actions)
        
        image_ap = process_img(image)
        
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        
        self._display.send(image) #display the image on screen
        
        return image_ap, reward, is_over #return the Experience tuple

In [7]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB to Grey Scale
    image = image[:224, :240] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [8]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [9]:
#game parameters
ACTIONS = 3 # possible actions: jump, do nothing
Key_num = 9
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 #We stack 4 frames

In [10]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [11]:
'''Call only once to init file structure
'''
#init_cache()

'Call only once to init file structure\n'

In [12]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

In [13]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(Key_num)
    #do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros(Key_num) # action at t
        #run, jump, stop
        
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                #action_index = random.randrange(ACTIONS)
                a_t = random.choice([[0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1,1], [0,0,0,0,0,0,1,1,0]])
                
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t_all = [[0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1,1], [0,0,0,0,0,0,1,1,0]]
                a_t = a_t_all[action_index-1]       # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], Key_num))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            #game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            #game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [14]:
'''import retro

def main():
    
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    
    game.restart()
    
    while True:
        obs, rew, done, info = game.get_score(game.Jump())
        game.render()
        if info['lives']<3:
            game.restart()
    game.end()'''

"import retro\n\ndef main():\n    \n    env_name = 'CircusCharlie-Nes'\n    game = Game(env_name)\n    \n    game.restart()\n    \n    while True:\n        obs, rew, done, info = game.get_score(game.Jump())\n        game.render()\n        if info['lives']<3:\n            game.restart()\n    game.end()"

In [15]:
'''import retro

#main function
def main(observe=True):
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    game.restart()
    
    clown = clown_agent(game)
    game_state = Game_state_(clown,game)    
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.restart()
'''

"import retro\n\n#main function\ndef main(observe=True):\n    env_name = 'CircusCharlie-Nes'\n    game = Game(env_name)\n    game.restart()\n    \n    clown = clown_agent(game)\n    game_state = Game_state_(clown,game)    \n    model = buildmodel()\n    try:\n        trainNetwork(model,game_state,observe=observe)\n    except StopIteration:\n        game.restart()\n"

In [16]:
import retro

#main function
def main(observe=True):
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    game.restart()
    
    clown = clown_agent(game)
    game_state = Game_state_(clown,game)    
    model = buildmodel()
    
    trainNetwork(model,game_state,observe=observe)
    obs, rew, done, info = game.get_score(game.get_action_space.sample())
        
    if info['lives']<3:
        game.restart()

In [ ]:
if __name__ == "__main__":
    main()

TIMESTEP 15000 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 1.4410533700819974
TIMESTEP 15001 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.04591715976331
TIMESTEP 15002 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.26250794772165
TIMESTEP 15003 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.428009738835875
TIMESTEP 15004 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 64.5923461923462
TIMESTEP 15005 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 65.48381758286365
TIMESTEP 15006 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 65.75999498291054
TIMESTEP 15007 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.59341975589417
TIMESTEP 15008 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0

fps: 65.43171819912016
TIMESTEP 15078 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.072545914667536
TIMESTEP 15079 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 65.2738845573245
TIMESTEP 15080 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.72325407507103
TIMESTEP 15081 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.371468286344545
TIMESTEP 15082 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.41802461419409
TIMESTEP 15083 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 64.6949654491609
TIMESTEP 15084 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.79086200185634
TIMESTEP 15085 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 61.30858170230804
TIMESTEP 15086 / STATE observe / EPSILON 0.0001 /

fps: 63.50387596899225
TIMESTEP 15156 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 64.48707738195907
TIMESTEP 15157 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 61.19230264213705
TIMESTEP 15158 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.799323241851205
TIMESTEP 15159 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.19482906691175
TIMESTEP 15160 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.679199605481415
TIMESTEP 15161 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 58.84512535600545
TIMESTEP 15162 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.119431279620855
TIMESTEP 15163 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.747920227920225
TIMESTEP 15164 / STATE observe / EPSILON 0.00

fps: 64.44150137508258
TIMESTEP 15234 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 61.367784979589445
TIMESTEP 15235 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.79368216183846
TIMESTEP 15236 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 64.98464589498474
TIMESTEP 15237 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.07346455527601
TIMESTEP 15238 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 64.91725739049683
TIMESTEP 15239 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 60.871705561360734
TIMESTEP 15240 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.66291759634504
TIMESTEP 15241 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 61.98906328515267
TIMESTEP 15242 / STATE observe / EPSILON 0.0001

fps: 59.73345486135836
TIMESTEP 15311 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.241581920903954
TIMESTEP 15312 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.22401547563576
TIMESTEP 15313 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.26334529629525
TIMESTEP 15314 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.12216850147301
TIMESTEP 15315 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.410246604059544
TIMESTEP 15316 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 60.088593450044414
TIMESTEP 15317 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 58.830268602286274
TIMESTEP 15318 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 61.781790864499406
TIMESTEP 15319 / STATE observe / EPSILON 0.0

...

TIMESTEP 15533 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 60.02409948910228
TIMESTEP 15534 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 57.198434452944944
TIMESTEP 15535 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 56.13887810688903
TIMESTEP 15536 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.88697402801377
TIMESTEP 15537 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 60.01293461153241
TIMESTEP 15538 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.72675752465169
TIMESTEP 15539 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 58.0052828831819
TIMESTEP 15540 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 64.4831116919056
TIMESTEP 15541 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 /

fps: 59.36820054070121
TIMESTEP 15607 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 55.80054812016071
TIMESTEP 15608 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.80684625866639
TIMESTEP 15609 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.77018856629751
TIMESTEP 15610 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.89254760833708
TIMESTEP 15611 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.1405084530675
TIMESTEP 15612 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 61.24412645104768
TIMESTEP 15613 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 55.984516611273506
TIMESTEP 15614 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 66.09887321724057
TIMESTEP 15615 / STATE observe / EPSILON 0.0001 /

fps: 54.16268288588437
TIMESTEP 15683 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 56.043613041154465
TIMESTEP 15684 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 57.79427610819451
TIMESTEP 15685 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.058899731057885
TIMESTEP 15686 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 56.35763137739677
TIMESTEP 15687 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 58.81294520163778
TIMESTEP 15688 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.07224060150376
TIMESTEP 15689 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 58.86825078246712
TIMESTEP 15690 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 60.468023758722104
TIMESTEP 15691 / STATE observe / EPSILON 0.000

fps: 59.48186175794878
TIMESTEP 15757 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 54.80099821001607
TIMESTEP 15758 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.39541421683469
TIMESTEP 15759 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 61.245020734770165
TIMESTEP 15760 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.90197960407919
TIMESTEP 15761 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.85287804283953
TIMESTEP 15762 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 55.04769404415046
TIMESTEP 15763 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.07720044508923
TIMESTEP 15764 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.72325407507103
TIMESTEP 15765 / STATE observe / EPSILON 0.0001 

fps: 57.375265037002585
TIMESTEP 15831 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 58.27445640847517
TIMESTEP 15832 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 60.6911400830572
TIMESTEP 15833 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 63.428009738835875
TIMESTEP 15834 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 55.71382649469336
TIMESTEP 15835 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 57.70680901999092
TIMESTEP 15836 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.86144208144118
TIMESTEP 15837 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 59.1672050670767
TIMESTEP 15838 / STATE observe / EPSILON 0.0001 / ACTION 2 / REWARD 0.0 / Q_MAX  0 / Loss  0
fps: 62.609962532280456
TIMESTEP 15839 / STATE observe / EPSILON 0.0001 